# Lightweight Networks and MobileNet

We have seen that complex networks require significant computational resources, such as GPU, for training, and also for fast inference. However, it turns out that a model with significanly smaller number of parameters in most cases can still be trained to perform resonably well. In other words, increase in the model complexity typically results in small (non-proportional) increase in the model performance.

We have observed this in the beginning of the module when training MNIST digit classification. The accuracy of simple dense model was not significanly worse than that of a powerful CNN. Increasing the number of CNN layers and/or number of neurons in the classifier allowed us to gain a few percents of accuracy at most.

This leads us to the idea that we can experiment with Lightweight network architectures in order to train faster models. This is especially important if we want to be able to execute our models on mobile devices.

This module will rely on the Cats and Dogs dataset that we have downloaded in the previous unit. First we will make sure that the dataset is available.

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchsummary import summary
import os

from pytorchcv import train, display_dataset, train_long, load_cats_dogs_dataset, validate, common_transform

In [2]:
if not os.path.exists('data/kagglecatsanddogs_3367a.zip'):
    !wget -P data -q http://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

dataset, train_loader, test_loader = load_cats_dogs_dataset()

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin

## MobileNet

In the previous unit, we have seen **ResNet** architecture for image classification. More lightweight analog of ResNet is **MobileNet**, which uses so-called *Inverted Residual Blocks*. Let's load pre-trained mobilenet and see how it works:

In [3]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()
print(model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

Using cache found in /home/vmuser/.cache/torch/hub/pytorch_vision_v0.6.0


Let's apply the model to our dataset and make sure that it works.

In [4]:
sample_image = dataset[0][0].unsqueeze(0)
res = model(sample_image)
print(res[0].argmax())

tensor(281)


**Exercise:** Compare the number of parameters in MobileNet and full-scale ResNet model.



## Using MobileNet for Transfer Learning

Now let's perform the same transfer learning process as in previous unit, but using MobileNet. First of all, let's freeze all parameters of the model:

In [7]:
for x in model.parameters():
    x.requires_grad = False

Then, replace the final classifier. We also transfer the model to our default training device (GPU or CPU):

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.classifier = nn.Linear(1280,2)
model = model.to(device)
summary(model,input_size=(3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 122, 122]             864
       BatchNorm2d-2         [-1, 32, 122, 122]              64
             ReLU6-3         [-1, 32, 122, 122]               0
            Conv2d-4         [-1, 32, 122, 122]             288
       BatchNorm2d-5         [-1, 32, 122, 122]              64
             ReLU6-6         [-1, 32, 122, 122]               0
            Conv2d-7         [-1, 16, 122, 122]             512
       BatchNorm2d-8         [-1, 16, 122, 122]              32
  InvertedResidual-9         [-1, 16, 122, 122]               0
           Conv2d-10         [-1, 96, 122, 122]           1,536
      BatchNorm2d-11         [-1, 96, 122, 122]             192
            ReLU6-12         [-1, 96, 122, 122]               0
           Conv2d-13           [-1, 96, 61, 61]             864
      BatchNorm2d-14           [-1, 96,

Now let's do the actual training:

In [15]:
train_long(model,train_loader,test_loader,loss_fn=torch.nn.CrossEntropyLoss(),epochs=1,print_freq=90)

Epoch 0, minibatch 0: train acc = 1.0, train loss = 0.00018727575661614537
Epoch 0, minibatch 90: train acc = 0.9495192307692307, train loss = 0.006199962490207546
Epoch 0, minibatch 180: train acc = 0.9526933701657458, train loss = 0.006265106122138092
Epoch 0, minibatch 270: train acc = 0.9556042435424354, train loss = 0.005903947397351705
Epoch 0, minibatch 360: train acc = 0.9552458448753463, train loss = 0.006147158773321854
Epoch 0, minibatch 450: train acc = 0.9548226164079823, train loss = 0.0062902259192287
Epoch 0, minibatch 540: train acc = 0.9539625693160814, train loss = 0.006862575597992225
Epoch 0 done, validation acc = 0.97695, validation loss = 0.0037557861328125


## Takeaway

Notice that MobileNet results in almost the same accuracy as VGG-16, and just slightly lower than full-scale ResNet. 

The main advantage of small models, such as MobileNet or ResNet-18 is that they can be used on mobile devices. [Here](https://pytorch.org/mobile/android/) is official example of using ResNet-18 on Android device, and [here](https://heartbeat.fritz.ai/pytorch-mobile-image-classification-on-android-5c0cfb774c5b) is similar example using MobileNet. 